In [154]:
## Librerias 
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import csv
import random
import kaggle

In [155]:
!chmod 600 /Users/alexandervalverde/.kaggle/kaggle.json

In [156]:
## Import data sets
train = pd.read_csv("/Users/alexandervalverde/Documents/Kaggle Competitions/Digit Recognizer/Datasets/train.csv")
test = pd.read_csv("/Users/alexandervalverde/Documents/Kaggle Competitions/Digit Recognizer/Datasets/test.csv")

In [157]:
## Select columns
train_y = train['label'].astype('float32')
train_x = train.drop(['label'],axis = 1).astype('int32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape

((42000, 784), (42000,), (28000, 784))

In [158]:
## Center data
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x / 255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [159]:
#One Hot Encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(42000, 10)

In [162]:
#Defining the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation = 'selu', input_shape=(28,28,1)),
  tf.keras.layers.Conv2D(32,(3,3),activation = 'selu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'selu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'selu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'selu',padding = 'Same'),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'selu',padding = 'Same'),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='selu'),
  #tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(256, activation='selu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_25 (Conv2D)          (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 12, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 conv2d_27 (Conv2D)          (None, 12, 12, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                

In [163]:
#Compiling and model training with batch size = 50, epochs = 20, and optimizer = adam
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001, 
            beta_1=0.9, 
            beta_2=0.999, 
            epsilon=1e-07,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 50, epochs = 20)

Epoch 1/20
840/840 [==============================] - 29s 35ms/step - loss: 0.3056 - accuracy: 0.9120
Epoch 2/20
840/840 [==============================] - 30s 36ms/step - loss: 0.1507 - accuracy: 0.9622
Epoch 3/20
840/840 [==============================] - 31s 37ms/step - loss: 0.1092 - accuracy: 0.9713
Epoch 4/20
840/840 [==============================] - 32s 38ms/step - loss: 0.1005 - accuracy: 0.9742
Epoch 5/20
840/840 [==============================] - 31s 37ms/step - loss: 0.0990 - accuracy: 0.9748
Epoch 6/20
840/840 [==============================] - 31s 37ms/step - loss: 0.0817 - accuracy: 0.9789
Epoch 7/20
840/840 [==============================] - 31s 37ms/step - loss: 0.0790 - accuracy: 0.9806
Epoch 8/20
840/840 [==============================] - 31s 37ms/step - loss: 0.0841 - accuracy: 0.9791
Epoch 9/20
840/840 [==============================] - 32s 38ms/step - loss: 0.0749 - accuracy: 0.9819
Epoch 10/20
840/840 [==============================] - 31s 37ms/step - loss: 0.082

In [164]:
## Save model
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


In [165]:
## Predict
results = model.predict(test_x)

# Select the index with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

875/875 [==============================] - 7s 8ms/step


In [166]:
## Submission

submission = pd.concat([pd.Series(range(1,28001), name = 'ImageId'), results], axis=1)

submission.to_csv("Digital_Recognition.csv",  index=False)

In [149]:
#Submitting the results on Kaggle
!kaggle competitions submit -c digit-recognizer -f Digital_Recognizer.csv -m "Proyect"

100%|█████████████████████████████████████████| 290k/290k [00:00<00:00, 341kB/s]
Successfully submitted to Digit Recognizer